In [1]:
import pandas as pd
import ast

In [2]:
!which python

/home/vscode/venv/bin/python


In [5]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# Download the dataset
# api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions', 'RAW_recipes.csv', unzip=True)

In [27]:
data = pd.read_csv('RAW_recipes.csv')
data.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


In [28]:
# Select only 'name' and 'steps' columns 
data = data[['name', 'steps', 'description', 'ingredients']]
# shape 
data.shape

In [29]:
# Count number of rows that contain 'please ignore' or 'none' string 
print(data[data['steps'].str.contains('please ignore')].shape)
print(data[data['steps'].str.contains('none')].shape)

(1, 4)
(103, 4)


In [30]:
# Remove rows that contain 'please ignore' or 'none' string
data = data[~data['steps'].str.contains('please ignore')]
data = data[~data['steps'].str.contains('none')]
data.shape

(231533, 4)

In [31]:
def clean_steps(data):
    # Convert the string representation of a list to an actual list
    data_list = ast.literal_eval(data)
    # Join the list elements into a single string
    return ', '.join(data_list)

data['steps'] = data['steps'].apply(clean_steps)
data['ingredients'] = data['ingredients'].apply(clean_steps)
data.head(2)

,name,steps,description,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil..."


In [32]:
# assemble lines: concatenate 'steps' with 'name'
lines = data.apply(lambda row: str(row['name']) + 
                   '; ' + 
                   str(row['steps']).replace("\n", ' ') + 
                   '; ' + 
                   str(row['description']) + 
                   '; ' + 
                   str(row['ingredients']), axis=1).tolist()
sorted(lines, key=len)[:10]


['cherry coke; blend and chill then serve; a friend; coke, kool-aid',
 'chicken test; cook it, eat it; i like chicken; banana, apple, milk',
 'delicious water; pour the water, delicious; water; water, lemon wedge',
 'twin peaks martini; shake and strain; nan; vodka, whiskey, triple sec',
 'deluxe scrambled eggs; cook in a pan; good eggs; egg, bacon, tomatoes, spinach',
 'tahini substitution; mix well; good to know.; tahini, peanut butter, sesame oil',
 'cape cod; shake with ice and serve on the rocks; nan; vodka, lime, cranberry juice',
 "polar blast; mix, enjoy; great on a winter's night.; brandy, rum, vanilla ice cream",
 'homemade sour cream with yogurt; mix and use; easy peasy!; plain yogurt, lemon juice',
 'maple milk; combine in a glass; very easy and tasty drink to make.; milk, maple syrup']

In [33]:
data.head(3)

,name,steps,description,ingredients
0,arriba baked winter squash mexican style,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,..."
1,a bit different breakfast pizza,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil..."
2,all in the kitchen chili,"brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to..."


In [34]:
# Clean the data. Use WordPunctTokenizer or something.
# Convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def clean_line(line):
    return ' '.join(tokenizer.tokenize(line.lower()))

lines = [clean_line(line) for line in lines]
sorted(lines, key=len)[:10]

['cherry coke ; blend and chill then serve ; a friend ; coke , kool - aid',
 'chicken test ; cook it , eat it ; i like chicken ; banana , apple , milk',
 'delicious water ; pour the water , delicious ; water ; water , lemon wedge',
 'twin peaks martini ; shake and strain ; nan ; vodka , whiskey , triple sec',
 'deluxe scrambled eggs ; cook in a pan ; good eggs ; egg , bacon , tomatoes , spinach',
 'tahini substitution ; mix well ; good to know .; tahini , peanut butter , sesame oil',
 'cape cod ; shake with ice and serve on the rocks ; nan ; vodka , lime , cranberry juice',
 'homemade sour cream with yogurt ; mix and use ; easy peasy !; plain yogurt , lemon juice',
 'maple milk ; combine in a glass ; very easy and tasty drink to make .; milk , maple syrup',
 "polar blast ; mix , enjoy ; great on a winter ' s night .; brandy , rum , vanilla ice cream"]

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

In [35]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - `UNK` represents absent tokens, 
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    Consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    for line in tqdm(lines):
        # YOUR CODE HERE
        tokens = line.split()
        tokens = [UNK]*(n-1) + tokens + [EOS]
        for i in range(len(tokens)-n+1):
            prefix = tuple(tokens[i:i+n-1])
            counts[prefix][tokens[i+n-1]] += 1

    return counts

counts = count_ngrams(lines, 3)



100%|██████████| 231533/231533 [00:46<00:00, 4989.31it/s]


In [41]:
# Check the counts for the prefix  ('I', 'love'). Limit the output to 10 elements
counts[('cup', 'of')].most_common(10)


[('the', 5036),
 ('water', 1300),
 ('sugar', 514),
 ('cheese', 489),
 ('flour', 412),
 ('milk', 328),
 ('batter', 273),
 ('hot', 273),
 ('coffee', 221),
 ('tea', 186)]

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [42]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)